In [ ]:
from telethon import TelegramClient
import json
import os
from dotenv import load_dotenv
import logging
import asyncio

# Load environment variables
load_dotenv()

# Setup logging
logging.basicConfig(filename='scraping.log', level=logging.INFO)

# Initialize Telegram client
api_id = os.getenv('API_ID')
api_hash = os.getenv('API_HASH')
client = TelegramClient('session_name', api_id, api_hash)

async def scrape_channel(channel):
    try:
        logging.info(f'Starting to scrape channel: {channel}')
        async for message in client.iter_messages(channel):
            if message.text:
                data = {
                    'id': message.id,
                    'text': message.text,
                    'date': message.date.isoformat(),
                    'media': message.media.url if message.media else None
                }
                date_folder = message.date.strftime('%Y-%m-%d')
                os.makedirs(f'../data/raw/telegram_messages/{date_folder}', exist_ok=True)

                file_path = f'../data/raw/telegram_messages/{date_folder}/{channel}.json'
                with open(file_path, 'a') as f:
                    json.dump(data, f)
                    f.write('\n')
                    logging.info(f'Scraped message {data["id"]} from {channel}')
            else:
                logging.info(f'Skipped message {message.id} (no text) from {channel}')
    except Exception as e:
        logging.error(f'Error scraping {channel}: {e}')

async def main():
    await client.start()  # Start the client
    channels = ['lobelia4cosmetics', 'tikvahpharma']
    for channel in channels:
        await scrape_channel(channel)

# Run the main function
if __name__ == "__main__":
    if not asyncio.get_event_loop().is_running():
        asyncio.run(main())
    else:
        asyncio.ensure_future(main())

Please enter your phone (or bot token): 